In [14]:
import csv
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, ElementClickInterceptedException

In [3]:
def get_url(position, location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [4]:
def save_data_to_file(records):
#     cols=['Job Title', 'Company', 'Location', 'Rating', 'Salary', 'Posting Date', 'Extract Date', 'Summary', 'URL']
#     my_df=pd.DataFrame(records,columns=cols)
#     my_df.to_csv('result.csv')
    with open('result.csv', 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Rating', 'Salary', 'Posting Date', 'Extract Date', 'Summary', 'URL'])
        writer.writerows(records)

In [5]:
def get_record(card):
    """Extract job data from single card"""
    try:
        jobs = card.find('h2','jobTitle').find_all('span')
        for job in jobs:
            if job.text.strip()=="new":
                continue
            else:
                job_title=job.text.strip()
    except AttributeError:
        job_title = ''
        
    try:
        company = card.find('span', 'companyName').text.strip()
    except AttributeError:
        company = ''
        
    try:
        location = card.find('div', 'companyLocation').text.strip()
    except AttributeError:
        location = ''
        
    try:
        job_li = card.find('div', 'job-snippet').find_all('li')
        job_summary = ''
        for li in job_li:
            job_summary+=li.text.strip("...")
    except AttributeError:
        job_summary = ''
        
    try:
        post_date = card.find('span', 'date').text.strip()
    except AttributeError:
        post_date = ''
        
    try:
        salary = card.find('div', 'salary-snippet').span.text.strip()
    except AttributeError:
        salary = ''
    
    try:
        rating = card.find('span', 'ratingNumber').span.text.strip()
    except AttributeError:
        rating = ''
        
    extract_date = datetime.today().strftime('%Y-%m-%d')
    atag=card['href']
    job_url = 'https://www.indeed.com' + atag
  
    return (job_title, company, location,rating, salary, post_date, extract_date, job_summary, job_url)
    

In [6]:
def get_page_records(cards, job_list, url_set):
    for card in cards:
        record = get_record(card)

        # add if job title exists and not duplicate
        if record[0] and record[-1] not in url_set:
            job_list.append(record)
            url_set.add(record[-1])

In [22]:
def main(position, location):

    scraped_jobs = []
    scraped_urls = set()
    
    url = get_url(position, location)
    
    driver = webdriver.Chrome()
    driver.get(url)  
    driver.implicitly_wait(5)     
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all('a',attrs={"class":"fs-unmask", "target":"_blank"})
        get_page_records(cards, scraped_jobs, scraped_urls)
        try:
            next_page=WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH, '//a[@aria-label="Next"]')))
            webdriver.ActionChains(driver).move_to_element(next_page).perform()
            next_page.click()
            time.sleep(3)
        except TimeoutException:
            break
        except ElementClickInterceptedException:
            driver.find_element_by_css_selector('.popover-x-button-close.icl-CloseButton').click()  # to handle job notification popup
            get_page_records(cards, scraped_jobs, scraped_urls)
            continue
    
    driver.quit()
    save_data_to_file(scraped_jobs)

In [70]:
main('machine learning', 'Seattle, WA')

In [72]:
main('computer vision', 'Seattle, WA')

In [73]:
main('nlp', 'Seattle, WA')

In [74]:
main('machine learning', 'New York, NY')

In [75]:
main('computer vision', 'New York, NY')

In [77]:
main('nlp', 'New York, NY')

In [78]:
main('machine learning', 'San Francisco, CA')

In [79]:
main('computer vision', 'San Francisco, CA')

In [25]:
main('nlp', 'San Francisco, CA')

In [30]:
import re
import pandas as pd
import string

In [35]:
a=["$132,400 - $211,600 a year","$50 - $65 an hour","$82,600 a year"]

In [58]:

for i in a:
    if re.findall("a year",i):
        tmp=re.findall(r'[0-9,]+',i)
        count=0
        sum=0
        for j in tmp:
            j=j.replace(',','')
            sum+=int(j)
            count+=1
        sum/=count
        
        

172000.0
82600.0


In [41]:
re.findall("a year",a[i])

['a year']